# 1. Выполнить установку библиотек
```
pip install -r requirements.txt 
```

In [ ]:
import os
import re
import time
import requests
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

# 2. Считываем файл с данными для парсинга
Пример txt файла:  
```txt
555424|444833|1|2|3|4
```

In [ ]:
with open("example_input.txt", "r", encoding="utf-8") as f:
    csv_id = f.read()

list_id = [str(i).strip() for i in csv_id.split("|") if str(i).strip() != ""]
print(f"Количество документов для парсинга: {len(list_id)}")

# 3. Выполняем запросы по каждому из патентов

В строке можно указать иную папку для записи HTML файлов
```
folder="html"
```

In [1]:
## Блок функций для выполнения HTTP запросов
def save_html(response, key: str, folder: str = "html1") -> str:
    """
    Сохраняет HTML-страницу в файл.
    :param response: объект Response из requests
    :param key: имя файла (используется для сохранения)
    :param folder: папка, куда сохранять файл (по умолчанию "htmls")
    :return: путь к сохраненному файлу
    """
    # Создаем папку, если она не существует
    os.makedirs(folder, exist_ok=True)

    # Путь к файлу
    file_path = os.path.join(folder, f"{key}.html")
    # Записываем HTML-контент в файл
    with open(file_path, "w", encoding="windows-1251") as file:
        file.write(response)

def send_request(key, url, folder):
    time.sleep(3.25)
    # Данные запроса (замени ID документа, если нужно)
    payload = {
        "DB": "RUTM",
        "DocNumber": f"{key}",  # номер документа
        "TypeFile": "html",
        "searchPar": "par_1",
        "searchParValue": f"{key}",
    }
    # Заголовки (имитация браузера)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
        "Referer": "https://new.fips.ru/registers-web/action?acName=clickRegister&regName=RUTM",
        "Origin": "https://new.fips.ru",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    # Отправка запроса
    try:
        response = requests.post(url=url, data=payload, headers=headers)
        # Проверяем статус ответа
        if response.status_code == 200:
            response.encoding = "windows-1251"  # Устанавливаем кодировку, так как сайт использует windows-1251
            html_content = response.text
            save_html(response.text, key, folder)
            return html_content
        else:
            return None
    except:
        return None

# URL для запроса
url = "https://new.fips.ru/registers-doc-view/fips_servlet"

for key in list_id:
    try:
        send_request(key, url, folder="html")
    except:
        print(key, end="\n")

    status           # Cтатус  
    last_change      # Последнее изменение статуса  
    reg_number       # Номер гос. регистрации  
    app_number       # Номер заявки  
    expiry_date      # Дата истечения исключительного права  
    prioritet        # Приоритет  
    app_date         # Дата подачи заявки  
    date_gov_reg     # Дата государственной регистрации  
    date_publish     # Дата публикации  
    right_holder     # Правообладатель  
    address          # Адрес для переписки  

# 4. Парсинг сформированных HTML документов

In [ ]:
def find_b(p):
    try:
        a_tag = p.find("b")  # Ищем <a> внутри
        if a_tag:
            return a_tag.get_text(strip=True)
    except:
        pass

def find_i(i_tag):
    try:
        b_tag = i_tag.find_next("b")  # Ищем следующий тег <b>
        if b_tag:
            return b_tag.get_text(strip=True)
    except:
        pass

def parse_html(html_text, index):
    # Парсим страницу
    soup = BeautifulSoup(html_text, "html.parser")

    status = None  # Cтатус
    last_change = None  # Последнее изменение статуса
    reg_number = None  # Номер гос. регистрации
    app_number = None  # Номер заявки
    expiry_date = None  # Дата истечения исключительного права
    prioritet = None  # Приоритет
    app_date = None  # Дата подачи заявки
    date_gov_reg = None  # Дата государственной регистрации
    date_publish = None  # Дата публикации
    right_holder = None  # Правообладатель
    address = None  # Адрес для переписки

    status_row = soup.find("tr", class_="Status")
    # Если такая строка найдена, извлекаем статус
    date_pattern = r"\b\d{2}\.\d{2}\.\d{4}\b|\b(\d{4}-\d{2}-\d{2}|\d{2}/\d{2}/\d{4}|\d{2}-\d{2}-\d{4})\b"
    if status_row:
        status_text = status_row.find("td").get_text(strip=True)
        # status = str(status_text).strip().replace("\n", " ").replace("\t", "")
        status = re.sub(r"[\t]", "", str(status_text))  # Удаляем табуляции
        status = re.sub(r"\n+", " ", str(status))  # Заменяем переводы строк на пробел
        status = status.strip()
        match = re.search(date_pattern, status_text)
        if match:
            last_change = match.group()
        else:
            last_change = status

    # Извлекаем номер государственной регистрации
    p_tags = soup.find_all("p", class_="bib")
    for p in p_tags:
        if "210" in p.get_text():  # Проверяем, содержит ли <p> текст "(210)"
            app_number = find_b(p)
        if "111" in p.get_text():
            reg_number = find_b(p)
        if "181" in p.get_text():
            expiry_date = find_b(p)
        if "220" in p.get_text():
            app_date = find_b(p)
        if "151" in p.get_text():
            date_gov_reg = find_b(p)
        if "450" in p.get_text():
            date_publish = find_b(p)
        if "732" in p.get_text():
            right_holder = find_b(p)

    # Поиск тега <i>, содержащего "Правообладатель:"
    for i_tag in soup.find_all("i"):
        # find_i(i_tag)
        if "Правообладатель:" in i_tag.get_text():
            right_holder = find_i(i_tag)
        if "Приоритет:" in i_tag.get_text():
            prioritet = find_i(i_tag)
        if "Дата государственной регистрации" in i_tag.get_text():
            date_gov_reg = find_i(i_tag)
        if "Дата публикации" in i_tag.get_text():
            date_publish = find_i(i_tag)
        if "Адрес для переписки" in i_tag.get_text():
            address = find_i(i_tag)
    for td_tag in soup.find_all("td"):
        if "Решение о регистрации" in td_tag:
            try:
                tr_tag = td_tag.find_next("td")  # Ищем следующий тег <b>
                if tr_tag:
                    date_gov_reg = tr_tag.get_text(strip=True)
            except:
                pass

        # <td>Решение о регистрации</td><td>14.10.2024</td>

    i = "Идентификатор"
    df.loc[df[i] == index, "status"] = status
    df.loc[df[i] == index, "last_change"] = (
        str(last_change).replace("(последнее изменение статуса: ", "").replace(")", "")
    )
    df.loc[df[i] == index, "reg_number"] = reg_number
    df.loc[df[i] == index, "app_number"] = app_number
    df.loc[df[i] == index, "expiry_date"] = expiry_date
    df.loc[df[i] == index, "prioritet"] = prioritet
    df.loc[df[i] == index, "app_date"] = app_date
    df.loc[df[i] == index, "date_gov_reg"] = date_gov_reg
    df.loc[df[i] == index, "date_publish"] = date_publish
    df.loc[df[i] == index, "right_holder"] = right_holder
    df.loc[df[i] == index, "address"] = address

def read_html(index):
    folder = "html"
    try:
        with open(f"{folder}/{index}.html", "r", encoding="windows-1251") as file:
            text = file.read().strip()
        if text == "Документ с данным номером отсутствует":
            df.loc[df["Идентификатор"] == index, "status"] = (
                "Документ с данным номером отсутствует"
            )
        elif text == "Возникла ошибка. Попробуйте повторить свой запрос позже":
            df.loc[df["Идентификатор"] == index, "status"] = (
                "Возникла ошибка. Попробуйте повторить свой запрос позже"
            )
        else:
            parse_html(html_text=text, index=index)
    except FileNotFoundError as e:
        print(f"FileNotFoundError : {index}")

# 5. Формируем результирующую таблицу

In [ ]:
data = {
    "Идентификатор": list_id,
    "status": "",
    "last_change": "",
    "reg_number": "",
    "app_number": "",
    "expiry_date": "",
    "prioritet": "",
    "app_date": "",
    "date_gov_reg": "",
    "date_publish": "",
    "right_holder": "",
    "address": "",
}

tqdm.pandas()
df = pd.DataFrame(data=data)
df["Идентификатор"].progress_apply(read_html)
df.to_excel("Результаты_парсинга.xlsx", index=False)

# Результат. Пример:
|Идентификатор|status|last_change|reg_number|app_number|expiry_date|prioritet|app_date|date_gov_reg|date_publish|right_holder|address|
|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|
|555424|Статус: действует (последнее изменение статуса: 23.10.2015)|23.10.2015|555424|2014721503|26.06.2024|26.06.2014|26.06.2014|22.10.2015|27.09.2023Бюл. № 19|Публичное ***|ул.Спасская|
|444833|Статус: действует (последнее изменение статуса: 24.09.2011)|24.09.2011|444833|2010707760|15.03.2020||15.03.2010|23.09.2011|17.05.2019|Публичное ***|ул. Большая|
